# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [2]:
import numpy as np

import pandas as pd

from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [3]:
try:
    df0 = pd.read_csv('geo_data_0.csv')
    df1 = pd.read_csv('geo_data_1.csv')
    df2 = pd.read_csv('geo_data_2.csv')
except:
    df0 = pd.read_csv('/datasets/geo_data_0.csv')
    df1 = pd.read_csv('/datasets/geo_data_1.csv')
    df2 = pd.read_csv('/datasets/geo_data_2.csv')
    
df0.info()
display(df0.head())

df1.info()
df1.head()
display(df1.head())

df2.info()
display(df2.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


Проверка на дубли

In [25]:
display(df0[df0.duplicated()])
display(df0[df1.duplicated()])
display(df0[df2.duplicated()])

,id,f0,f1,f2,product


,id,f0,f1,f2,product


,id,f0,f1,f2,product


In [26]:
#Выделение признаков и целевого признака
features0 = df0.drop(['id', 'product'], axis=1)
features1 = df1.drop(['id', 'product'], axis=1)
features2 = df2.drop(['id', 'product'], axis=1)

target0 = df0['product']
target1 = df1['product']
target2 = df2['product']

features_train0, features_valid0, target_train0, target_valid0 = (
    train_test_split(features0, target0, test_size=0.25, random_state=12345)
)
features_train1, features_valid1, target_train1, target_valid1 = (
    train_test_split(features1, target1, test_size=0.25, random_state=12345)
)
features_train2, features_valid2, target_train2, target_valid2 = (
    train_test_split(features2, target2, test_size=0.25, random_state=12345)
)

## Обучение и проверка модели

In [46]:
def predict_region(model, features_train, target_train, features_valid, target_valid, num_reg):
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    predictions = pd.Series(predictions, index=target_valid.index)
    mse = mean_squared_error(target_valid, predictions)
    rmse = mse ** 0.5
    print(f'Средний запас сырья (предсказания, регион {num_reg}): {(predictions.sum()/len(predictions)).round(0)}')
    print(f'RMSE {num_reg}: {rmse} \n')
    return predictions

model0 = LinearRegression()
predictions0 = predict_region(model0, features_train0, target_train0, features_valid0, target_valid0, 0)

model1 = LinearRegression()
predictions1 = predict_region(model1, features_train1, target_train1, features_valid1, target_valid1, 1)

model2 = LinearRegression()
predictions2 = predict_region(model2, features_train2, target_train2, features_valid2, target_valid2, 2)

Средний запас сырья (предсказания, регион 0): 93.0
RMSE 0: 37.5794217150813 

Средний запас сырья (предсказания, регион 1): 69.0
RMSE 1: 0.893099286775617 

Средний запас сырья (предсказания, регион 2): 95.0
RMSE 2: 40.02970873393434 



Если анализировать предсказания по добыче в разных регионах, то средний уровень добычи со скважины наибольший в регионе "Регион 2", и составляет 95 тыс. баррелей со скважины (корень из среднеквадратичной ошибки - 40 тыс. баррелей).

## Подготовка к расчёту прибыли

In [28]:
#При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
Sample_size = 500
Count_best = 200
#Бюджет на разработку скважин в регионе — 10 млрд рублей.
Budget = 1e10
print(Budget)
#При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
Revenue_per_bar = 450_000
#После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.
Loss_prob = 0.025
#Объем для достижения точки безубыточности
Min_volume_limit = round(Budget / Revenue_per_bar / Count_best)

print("Минимальное среднее значение сырья для достижения точки безубыточности:", Min_volume_limit)

10000000000.0
Минимальное среднее значение сырья для достижения точки безубыточности: 111


Видно, что в среднем уровень безубыточности достигается при среднем объеме сырья в скважине - 111 тыс. бар. Если анализировать по среднему, то ни один из регионов не подходит для разработки. Т.е. 100% гарантии нет. Однако, нам на помощь приходит машинное обучение, которое сможет подобрать регион с большей вероятностью получения прибыли и меньшими рисками убытков. К тому же, цены на нефть могут вырасти, как бывало неоднократно в истории.

## Расчёт прибыли и рисков 

In [49]:
#Количество итераций для bootstrap
N_iter = 1000
state = np.random.RandomState(12345)

#Функция для расчет прибыли
def revenue(predictions, target, count):
    predictions.sort_values(ascending=False, inplace=True)
    selected = target[predictions.index][:count]
    return revenue_per_bar * selected.sum() - budget

def bootstrap_region(predictions, target, n_reg):
    values = []
    loss_count = 0
    pred = pd.Series(predictions)
    for i in range(N_iter):
        pred_subsample = pred.sample(n=Sample_size, random_state=state, replace=True)
        cur_revenue = revenue(pred_subsample, target, Count_best)
        values.append(cur_revenue)
        loss_count += (cur_revenue < 0)

    values = pd.Series(values)
    mean = values.mean().round(0)
    print(f'Средняя прибыль (регион {n_reg}): {mean:,}')
    print(f'Вероятность получения убытка (регион {n_reg}): {round(100 * loss_count/N_iter, 4)}%')
    low = values.quantile(0.025).round(0)
    high = values.quantile(0.975).round(0)
    print(f'95% доверительный интервал (регион {n_reg}): ({low:,} {high:,})  \n')
    

bootstrap_region(predictions0, target_valid0, 0)
bootstrap_region(predictions1, target_valid1, 1)
bootstrap_region(predictions2, target_valid2, 2)

Средняя прибыль (регион 0): 396,164,985.0
Вероятность получения убытка (регион 0): 6.9%
95% доверительный интервал (регион 0): (-111,215,546.0 909,766,942.0)  

Средняя прибыль (регион 1): 461,155,817.0
Вероятность получения убытка (регион 1): 0.7%
95% доверительный интервал (регион 1): (78,050,811.0 862,952,060.0)  

Средняя прибыль (регион 2): 392,950,475.0
Вероятность получения убытка (регион 2): 6.5%
95% доверительный интервал (регион 2): (-112,227,625.0 934,562,915.0)  



Выше всего потенциальная средняя прибыль в регионе "Регион 1", и составляет 461,155,817 руб.  с 95% доверительным интервалом (78,050,811.0 862,952,060.0)

## Выводы

Проведен выбор наиболее оптимального региона для бурения новых скважин путем анализа данных по уже функционирующим скважинам.
Более всего подходит "Регион 1", т. к. потенциально полученная прибыль при бурении в нем 200 скважин составляет 461,155,817 руб. с 95% доверительным интервалом (78,050,811.0 862,952,060.0), а вероятность получения убытка - 0.7%